## Build user table containing hubspot contact informations

In [1]:
import pandas as pd
import os

In [2]:
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",
}

## Load input data EE Faculty Delivera full names

EE data data: $ cwd+EE Faculty Delivra full names.xlsx

In [3]:
#import data
cwd = os.getcwd()
print(cwd)
temp = pd.read_excel('EE_delivra.xlsx')

C:\Users\henry\Desktop\PhD Research\Nanohub RA\nanohub_salesforce_integ-master\delivera_table_generation


In [4]:
temp.head(2)

,RecipientID,MemberID,engagement,UserName,Domain,EmailAddr_,MailingStatus,SendTry,Full Name
0,19265625052,1558002285,0.0000,mchamanz,andrew.cmu.edu,mchamanz@andrew.cmu.edu,Success,1,maysam chamanzar
1,19265625108,1558002287,0.4167,qingli2,andrew.cmu.edu,qingli2@andrew.cmu.edu,Success,1,qing li


### Split by mailingstatus

In [8]:
# find row indices that have unsuccessful mailing status
badindex = []
goodindex = []
for i,j in enumerate(temp['MailingStatus']):
    if j != 'Success':
        badindex.append(i)
    else:
        goodindex.append(i)

In [9]:
# extract the entries with failure mailing status and save the result
b_ee_df = temp.iloc[badindex,:]
b_ee_df.to_csv(cwd+'\ee_delivra_failures.csv')

In [10]:
# reset the temporary holding dataframe
temp = temp.iloc[goodindex,:]

In [11]:
display(temp.head(2))

,RecipientID,MemberID,engagement,UserName,Domain,EmailAddr_,MailingStatus,SendTry,Full Name
0,19265625052,1558002285,0.0000,mchamanz,andrew.cmu.edu,mchamanz@andrew.cmu.edu,Success,1,maysam chamanzar
1,19265625108,1558002287,0.4167,qingli2,andrew.cmu.edu,qingli2@andrew.cmu.edu,Success,1,qing li


### Filtering for salesforce import

In [12]:
# filtering for 'engagement','EmailAddr_','MailingStatus','Full Name'
t_list = []
t_df = pd.DataFrame() #['engagement','EmailAddr_','MailingStatus','Full Name']
t_df['Engagement'] = temp.loc[:,'engagement']
t_df['Email'] = temp.loc[:,'EmailAddr_']
t_df['fullname'] = temp.loc[:,'Full Name']

In [22]:
# split full name into first and last name
def first_name(this_name):
    name_list = this_name.split(' ')

    return pd.Series([name_list[0]])

def last_name(this_name):
    name_list = this_name.split(' ')
    
    return pd.Series([name_list[-1]])


t_df['firstname'] = t_df['fullname'].apply(first_name)
t_df['lastname'] = t_df['fullname'].apply(last_name)
t_df['Name'] = t_df['firstname']+' '+t_df['lastname']
t_df['contact_type'] = 'EE2019list'
t_df['lead_source'] = 'EE Delivra Engagement'

In [23]:
display(t_df.head(2))
display(t_df.tail(2))

,Engagement,Email,fullname,firstname,lastname,Name,contact_type,lead_source
0,0.0000,mchamanz@andrew.cmu.edu,maysam chamanzar,maysam,chamanzar,maysam chamanzar,EE2019list,EE Delivra Engagement
1,0.4167,qingli2@andrew.cmu.edu,qing li,qing,li,qing li,EE2019list,EE Delivra Engagement


,Engagement,Email,fullname,firstname,lastname,Name,contact_type,lead_source
463,0.4167,mark.reed@yale.edu,mark a,mark,a,mark a,EE2019list,EE Delivra Engagement
464,0.8333,zypman@yu.edu,fredy zypman,fredy,zypman,fredy zypman,EE2019list,EE Delivra Engagement


## Import salesforce contacts - filter the data for contacts and leads

In [15]:
# Obtain contacts metadata from Salesforce
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
contact_metadata = db_s.get_obj_metadata('Contact')

Obtained Salesforce access token ...... True


In [16]:
import json

# get all fields
field_list = json.loads(contact_metadata)['fields']

# list of available field API names
available_fields = list()

for this_field in field_list:
    this_name = this_field['name']
    
    if 'Name' == this_name:
        print(True)
    
    if len(this_name) > 3:
        if this_name[-3:] == '__c':
            # this is a custom-made field
            available_fields.append(this_name[:-3])
            
# display
display(available_fields[:5])

True


['Type',
 'nanoHUB_user_ID',
 'nanoHUB_username',
 'Organization_Type',
 'nanoHUB_registration_date']

In [17]:
# salesforce queries for contact data
sf_contact_email_df = db_s.query_data('SELECT Id, Email FROM Contact')

[Success] Bulk job creation successful. Job ID = 7505w00000O5R17AAF
{"id":"7505w00000O5R17AAF","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-07-22T19:41:56.000+0000","systemModstamp":"2020-07-22T19:41:57.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000O5R17AAF","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-07-22T19:41:56.000+0000","systemModstamp":"2020-07-22T19:41:58.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000O5R17AAF","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-07-22T19:41:56

In [18]:
sf_contact_email_df.head(2)

,Email,Id
0,nkissebe@gmail.com,0035w000031Vsp1AAC
1,support@nanohub.org,0035w000031Vsp2AAC


In [24]:
# basically, if their email already exists as a contact, then drop them
contact_index = []
sf_id_contact = []
lead_index = []

sf_contacts = sf_contact_email_df['Email'].tolist()
for i,j in enumerate(t_df['Email']):
    if j in sf_contacts:
        contact_index.append(i)
        sf_id_contact.append(sf_contacts.index(j))
    else:
        lead_index.append(i)


In [25]:
# for contacts, update their contacts page
contacts_df = pd.DataFrame()
contacts_df['Engagement__c'] = t_df['Engagement']
contacts_df['LeadSource'] = t_df['lead_source']
contacts_df['Email'] = t_df['Email']

contacts_df = contacts_df.iloc[contact_index,:]
contacts_df = contacts_df.reset_index(drop=True)

ttc = sf_contact_email_df['Id'][sf_id_contact]
ttc = ttc.reset_index(drop=True)
contacts_df['Id'] = ttc


In [26]:
contacts_df.head(2)

,Engagement__c,LeadSource,Email,Id
0,0.4167,EE Delivra Engagement,stephen.goodnick@asu.edu,0035w000034JKc2AAG
1,0.4167,EE Delivra Engagement,t.thornton@asu.edu,0035w000034Joy5AAC


In [31]:
# for leads, create lead_df
lead_df = pd.DataFrame()
lead_df['Name'] = t_df['Name']
lead_df['Email'] = t_df['Email']
lead_df['Engagement__c'] = t_df['Engagement']
lead_df['LeadSource'] = t_df['lead_source']
lead_df['Contact_Type__c'] = t_df['contact_type']
lead_df = lead_df.reset_index(drop=True)
display(lead_df.head(2))

,Name,Email,Engagement__c,LeadSource,Contact_Type__c
0,maysam chamanzar,mchamanz@andrew.cmu.edu,0.0000,EE Delivra Engagement,EE2019list
1,qing li,qingli2@andrew.cmu.edu,0.4167,EE Delivra Engagement,EE2019list


## Match data with Salesforce format [CONTACTS]

In [32]:
external_id = 'Id'
object_id = 'Contact'

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id 
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [33]:
db_s.send_data(contacts_df)

[Success] Bulk job creation successful. Job ID = 7505w00000O5QyxAAF
hello
[Success] CSV upload successful. Job ID = 7505w00000O5QyxAAF
[Success] Closing job successful. Job ID = 7505w00000O5QyxAAF


In [34]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000O5QyxAAF',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-07-22T19:50:29.000+0000',
 'systemModstamp': '2020-07-22T19:50:30.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [35]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


## Match data with Salesforce format [LEADS]

In [36]:
external_id = 'Email'
object_id = 'Lead'
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id 
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [39]:
db_s.send_data(lead_df)

[Success] Bulk job creation successful. Job ID = 7505w00000O5R4LAAV
hello
[Success] CSV upload successful. Job ID = 7505w00000O5R4LAAV
[Success] Closing job successful. Job ID = 7505w00000O5R4LAAV


In [40]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000O5R4LAAV',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-07-22T19:53:39.000+0000',
 'systemModstamp': '2020-07-22T19:53:40.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Email',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [41]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
